# NÃO ESQUECER DE BAIXAR OS DADOS NECESSÁRIOS ANTES DE QUALQUER COISA.

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.2
✔ purrr     1.2.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


### Importando os dados do Scopus

In [ ]:
#df <- read_csv('scopus_data.csv')
df <- read_csv('scopus_data2.csv')

Rows: 316 Columns: 45
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (36): Authors, Author full names, Author(s) ID, Title, Source title, Vol...
dbl  (6): Year, Page start, Page end, Cited by, Conference code, PubMed ID
lgl  (3): Molecular Sequence Numbers, Tradenames, Manufacturers

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
'Dimensão de df:'
dim(df)
''
'Nomes das colunas de df:'
df |>
names()

[1] "Dimensão de df:"

[1] 316  45

[1] ""

[1] "Nomes das colunas de df:"

[1] "Authors"                       "Author full names"            
 [3] "Author(s) ID"                  "Title"                        
 [5] "Year"                          "Source title"                 
 [7] "Volume"                        "Issue"                        
 [9] "Art. No."                      "Page start"                   
[11] "Page end"                      "Cited by"                     
[13] "DOI"                           "Link"                         
[15] "Affiliations"                  "Authors with affiliations"    
[17] "Abstract"                      "Author Keywords"              
[19] "Index Keywords"                "Molecular Sequence Numbers"   
[21] "Chemicals/CAS"                 "Tradenames"                   
[23] "Manufacturers"                 "Funding Details"              
[25] "Funding Texts"                 "References"                   
[27] "Correspondence Address"        "Editors"                      
[29] "Publisher"                     "Sponsors"                     
[31] "Conference name"               "Conference date"              
[33] "Conference location"           "Conference code"              
[35] "ISSN"                          "ISBN"                         
[37] "CODEN"                         "PubMed ID"                    
[39] "Language of Original Document" "Abbreviated Source Title"     
[41] "Document Type"                 "Publication Stage"            
[43] "Open Access"                   "Source"                       
[45] "EID"

### Converter campos de texto para minúsculas (para facilitar busca por palavras-chave)


In [ ]:
df_lower <- df |>
  mutate(
    abstract_lower = str_to_lower(Abstract),
    keywords_lower = str_to_lower(`Author Keywords`)
  )

### Função auxiliar: detecta se algum termo de IA está presente

In [ ]:
# palavras-chave de IA
palavras_ia <- c("machine learning", "artificial intelligence", "deep learning", "neural network",
                 "random forest", "support vector", "svm", "decision tree",
                 "gradient boosting", "xgboost", "k-means", "unsupervised learning", "supervised learning")

In [ ]:
#funcao para encontrar em qualquer texto algum dos elementos de palavras_ia e retornar TRUE/FALSE.
detecta_ia <- function(texto) {
  if (is.na(texto) || !nzchar(texto )) return(FALSE)
  any(str_detect(texto, fixed(palavras_ia, ignore_case = TRUE)))
}

#eu já tinha tornado as letras minúsculas, mas por via das dúvidas, optei por usar o 'ignore_case = TRUE'.
#só para deixar claro, o fixed é usado quando não estamos usando o padrão regex.

In [ ]:
#funcao para encontrar em qualquer texto algum dos elementos da palavra_ia e retornar quais termos foram encontrados.
termos_ia_encontrados <- function(texto) {
  if (is.na(texto) || !nzchar(texto)) return(character(0)) #adiante, uma aplico a função str_detect onde eu quero para todos os elementos de palavras_ia, usando o vaplly
  palavras_ia[vapply(palavras_ia, function(p) str_detect(texto, fixed(p, ignore_case = TRUE)), logical(1))]
}


In [ ]:
#funcao para encontrar em qualquer texto algum dos elementos da palavra_ia e retornar quais termos foram encontrados em forma de string.
termos_ia_str <- function(texto) {
  if (is.na(texto) || !nzchar(texto)) return('vazio')

  #adiante, uma aplico a função str_detect onde eu quero para todos os elementos de palavras_ia, usando o vaplly
  termos <- palavras_ia[vapply(palavras_ia, function(p) str_detect(texto, fixed(p, ignore_case = TRUE)), logical(1))]

  if (length(termos)==0){
    'nao consta'
  } else {
    paste(termos, collapse=' / ')
  }
}


In [ ]:
numero=16
termos_ia_encontrados(df_lower$keywords_lower[numero])
termos_ia_str(df_lower$keywords_lower[numero])

[1] "machine learning" "neural network"

[1] "machine learning / neural network"

numero=6
map_lgl(df_lower$abstract_lower[1:10], detecta_ia)
map_lgl(df_lower$keywords_lower[1:10], detecta_ia)


 palavras_ia[
    vapply(
      palavras_ia,
      function(p) str_detect(df_lower$keywords_lower[numero], fixed(p, ignore_case = TRUE)),
      logical(1)
    )
  ]

#versão alternativa para detectar_ia no texto
detecta_ia <- function(texto) {
  if (is.na(texto) || !nzchar(texto)) return(FALSE)

  any(vapply(
    palavras_ia,
    function(p) str_detect(texto, fixed(p, ignore_case = TRUE)),
    logical(1)
  ))
}

### Criar coluna "usa_IA" com base no abstract e nas palavras-chave


In [ ]:
df_ia <- df_lower |>
  mutate(
    usa_IA = case_when(
      map_lgl(abstract_lower, detecta_ia) | map_lgl(keywords_lower, detecta_ia) ~ "sim", #map_lgl é um mapeamento que retorna operador lógico. Estamos percorrendo os abstracts de todos os artigos, isso é, de linha em linha.
      !is.na(abstract_lower) ~ "nao", # se não na, nao
      TRUE ~ "talvez" #se for na, não dá pra afirmar.
    ),
    IA_keywords = map_chr(keywords_lower,termos_ia_str),
    IA_abstract = map_chr(abstract_lower,termos_ia_str)
  )

In [ ]:
df_ia |>
  mutate(row_id = row_number()) |>
  select(row_id,Title,usa_IA,IA_keywords,IA_abstract)

row_id,Title,usa_IA,IA_keywords,IA_abstract
<int>,<chr>,<chr>,<chr>,<chr>
1,Causal Digital Twins for cyber–physical security in water systems: A framework for robust anomaly detection,nao,nao consta,nao consta
2,Building smart urban water systems from source to tap: Applications and challenges of deep learning,sim,neural network,deep learning
3,AI-Based Sensor Optimization and Anomaly Detection in Water Distribution Networks,sim,nao consta,random forest
4,A Bibliometric and Network Analysis of Digital Twins and BIM in Water Distribution Systems,nao,nao consta,nao consta
5,Learning physics and temporal dependencies: real-time modeling of water distribution systems via Kolmogorov–Arnold attention networks,nao,vazio,nao consta
6,Detecting cyberattacks based on deep neural network approaches in industrial control systems,sim,neural network,neural network
7,Detection of water pipeline leakages using high-resolution satellite imagery and auxiliary ground data in Cyprus: Case study of community councils of Doros and Monagri,nao,nao consta,nao consta
8,Multi-objective optimization of pressure sensor placement for leakage location and network calibration,nao,nao consta,nao consta
9,A novel data cleaning and completion framework for NB-IoT water meters in district metered areas based on purification ratio coefficients,nao,nao consta,nao consta


### Rotina para verificar manualmente o porquê de determinado artigo ter sido classificado como usa_IA = 'sim'

In [ ]:
numero=16
df_ia$Title[numero]
df_ia$keywords_lower[numero]
termos_ia_encontrados(df_ia$abstract_lower[numero])
termos_ia_encontrados(df_ia$keywords_lower[numero])

[1] "WatAI: AI-Based System for Real-Time Flow Monitoring and Demand Prediction in Water Networks"

[1] "artificial neural networks; flow prediction; iot; machine learning; real-time monitoring; smart water management"

[1] "machine learning" "neural network"

[1] "machine learning" "neural network"

### Ver alguns aspectos


In [ ]:
'Tabela contendo a contagem de artigos que usam IA e que não usam:'
df_ia |>
  dplyr::count(usa_IA)
'------------------------------------------------------------------------------'
'Dimensao novo dataframe:'
dim(df_ia)
'------------------------------------------------------------------------------'
'Nomes das colunas novas:'
df |>
  names()

[1] "Tabela contendo a contagem de artigos que usam IA e que não usam:"

usa_IA,n
<chr>,<int>
nao,161
sim,155


[1] "------------------------------------------------------------------------------"

[1] "Dimensao novo dataframe:"

[1] 316  50

[1] "------------------------------------------------------------------------------"

[1] "Nomes das colunas novas:"

[1] "Authors"                       "Author full names"            
 [3] "Author(s) ID"                  "Title"                        
 [5] "Year"                          "Source title"                 
 [7] "Volume"                        "Issue"                        
 [9] "Art. No."                      "Page start"                   
[11] "Page end"                      "Cited by"                     
[13] "DOI"                           "Link"                         
[15] "Affiliations"                  "Authors with affiliations"    
[17] "Abstract"                      "Author Keywords"              
[19] "Index Keywords"                "Molecular Sequence Numbers"   
[21] "Chemicals/CAS"                 "Tradenames"                   
[23] "Manufacturers"                 "Funding Details"              
[25] "Funding Texts"                 "References"                   
[27] "Correspondence Address"        "Editors"                      
[29] "Publisher"                     "Sponsors"                     
[31] "Conference name"               "Conference date"              
[33] "Conference location"           "Conference code"              
[35] "ISSN"                          "ISBN"                         
[37] "CODEN"                         "PubMed ID"                    
[39] "Language of Original Document" "Abbreviated Source Title"     
[41] "Document Type"                 "Publication Stage"            
[43] "Open Access"                   "Source"                       
[45] "EID"

### Encontrar duplicatas

Estabelecer uma função que reconheça todos os duplicados, não os duplicados a partir do segundo elemento que se repete, como o 'duplicated()'

In [ ]:
duplicated_any <- function(x) duplicated(x) | duplicated(x, fromLast = TRUE)

Aplicando a função para os DOI

In [ ]:
df_duplicatas_doi <- df_ia |>
  mutate(
    doi_clean = str_to_lower(str_squish(DOI)),
    duplicated_doi = case_when(
      is.na(doi_clean) ~ "no",  # se não tem DOI, não marca como duplicado
      duplicated_any(doi_clean) ~ "yes",
      TRUE ~ "no"
  )
  )

Aplicando a função para os Title

In [ ]:
df_duplicatas_doi_title <- df_duplicatas_doi |>
  mutate(
    title_clean = Title |>
      str_to_lower() |>
      str_squish()
  ) |>
  mutate(
    duplicated_title = if_else(duplicated_any(title_clean), "yes", "no")
  )

  #str_squish() é para limpar espaços desnecessários entre os textos, como no inicio e no fim, e espaços duplos.

Analisando as duplicatas

In [ ]:
# Contagem de duplicados
df_duplicatas_doi_title |>
  dplyr::count(duplicated_doi)

df_duplicatas_doi_title |>
  dplyr::count(duplicated_title)

duplicated_doi,n
<chr>,<int>
no,316


duplicated_title,n
<chr>,<int>
no,308
yes,8


In [ ]:
# Visualizar os duplicados
df_duplicatas_doi_title |>
  filter(duplicated_title=='yes') |>
  select(Title,DOI,Authors,Year)

Title,DOI,Authors,Year
<chr>,<chr>,<chr>,<dbl>
Development of Water Quality Analysis for Anomaly Detection and Correlation with Case Studies in Water Supply Systems,10.3390/electronics14101933,"Hanifa, R.; Cha, M.; Kang, W.; Yu, J.; Kim, K.-J.; Yun, Y.-M.; Kim, S.",2025
"9th International Building Physics Conference, IBPC 2024",NA,NA,2025
"9th International Building Physics Conference, IBPC 2024",NA,NA,2025
"9th International Building Physics Conference, IBPC 2024",NA,NA,2025
"9th International Building Physics Conference, IBPC 2024",NA,NA,2025
Development of Water Quality Analysis for Anomaly Detection and Correlation with Case Studies in Water Supply Systems,10.1109/PLATCON63925.2024.10830723,"Cha, M.; Kang, W.; Yun, Y.-M.; Yu, J.; Kim, K.-J.; Kim, S.",2024
Anomaly and attack detection in supervisory control networks for cyber-physical systems,NA,"del Prete, E.; Pera, F.; Faramondi, L.; Fioravanti, C.; Guarino, S.; Oliva, G.; Setola, R.",2020
Anomaly and attack detection in supervisory control networks for cyber-physical systems,10.3850/978-981-14-8593-0_4315-cd,"del Prete, E.; Pera, F.; Faramondi, L.; Fioravanti, C.; Guarino, S.; Oliva, G.; Setola, R.",2020


In [ ]:
'Se tiver 237 (316 no caso do Scopus2) linhas, é pq nenhum título estava NA'
df_duplicatas_doi_title |>
  filter(!is.na(title_clean)) |>
  dim()

[1] "Se tiver 237 (316 no caso do Scopus2) linhas, é pq nenhum título estava NA"

[1] 316  54

NÃO EXCLUIR OS DUPLICADOS SEM OLHAR

### Criar um dataframe onde mantemos a coluna de uso_IA e removemos duplicatas e artigos sem título

In [ ]:
df_ia_sem_duplicatas <- df_ia |>
  mutate(
    doi_clean = DOI |> str_to_lower() |> str_squish(),
    title_clean = Title |> str_to_lower() |> str_squish()
  ) |>
  filter(
    !duplicated(doi_clean) | is.na(doi_clean),
    !duplicated(title_clean)
  )

  #perceba que agora usei duplicated, pois se tivesse usado duplicated_any eu apagaria o n duplicados, e não n-1.

In [ ]:
paste0('Esta é a dimensão do dataframe sem duplicatas de titulo e DOI: ',nrow(df_ia_sem_duplicatas),'x',ncol(df_ia_sem_duplicatas))
paste0('Foram eliminados ', nrow(df_ia)-nrow(df_ia_sem_duplicatas),' artigos duplicados')

[1] "Esta é a dimensão do dataframe sem duplicatas de titulo e DOI: 311x52"

[1] "Foram eliminados 5 artigos duplicados"

### Exportar um .csv para que eu possa analisar e preencher manualmente no excel de acordo com os critérios de inclusão/exclusão.

write_csv(df_ia_sem_duplicatas, 'Artigos_IA_semDuplicatas.csv')

# CRITÉRIOS DE INCLUSÃO E EXLUSÃO:
Nesta etapa, analisarei se os artigos cumprem os critérios de inclusão e criarei colunas para identificá-los.

Criarei um novo dataframe mais enxuto, somente com as colunas necessárias, depois farei um full_join para juntar com o df_ia_sem_duplicatas.

### Criterio de Decisão: CD_Year_2000_2025


In [ ]:
df_cd1 <- df_ia_sem_duplicatas |>
  select(Title, Year, Abstract) |>
  mutate(CD_Year_2000_2025 = ifelse(Year >= 2000, 'sim', 'nao'))


In [ ]:
# ESTUDOS DE 1999 - 2026.

df_cd1 |>
  distinct(Year)

Year
<dbl>
2026
2025
2024
2023
2022
2021
2020
2019
2018


### Criterio de Decisao: CD_English

In [ ]:
install.packages('textcat')
library(textcat)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘tau’, ‘slam’




In [ ]:
df_cd1 |>
  mutate(CD_Idioma = textcat(Abstract)) |>
  pull(CD_Idioma) |>
  unique()

[1] "english"  "romanian"

In [ ]:
df_cd1 |>
  mutate(CD_Idioma = textcat(Abstract)) |>
  filter(CD_Idioma=='romanian')

#Na verdade, é inglês, mas o pacote identificou errado.

Title,Year,Abstract,CD_Year_2000_2025,CD_Idioma
<chr>,<dbl>,<chr>,<chr>,<chr>
Real-time in-network distribution system monitoring to improve operational efficiency,2011,[No abstract available],sim,romanian


In [ ]:
df_cd2 <- df_cd1 |>
  mutate(CD_English='sim')

### Criterio de Decisão: CD_RevisaoPares

FAZER ESSA FILTRAGEM NO SCOPUS

In [ ]:
names(df_ia_sem_duplicatas)

[1] "Authors"                       "Author full names"            
 [3] "Author(s) ID"                  "Title"                        
 [5] "Year"                          "Source title"                 
 [7] "Volume"                        "Issue"                        
 [9] "Art. No."                      "Page start"                   
[11] "Page end"                      "Cited by"                     
[13] "DOI"                           "Link"                         
[15] "Affiliations"                  "Authors with affiliations"    
[17] "Abstract"                      "Author Keywords"              
[19] "Index Keywords"                "Molecular Sequence Numbers"   
[21] "Chemicals/CAS"                 "Tradenames"                   
[23] "Manufacturers"                 "Funding Details"              
[25] "Funding Texts"                 "References"                   
[27] "Correspondence Address"        "Editors"                      
[29] "Publisher"                     "Sponsors"                     
[31] "Conference name"               "Conference date"              
[33] "Conference location"           "Conference code"              
[35] "ISSN"                          "ISBN"                         
[37] "CODEN"                         "PubMed ID"                    
[39] "Language of Original Document" "Abbreviated Source Title"     
[41] "Document Type"                 "Publication Stage"            
[43] "Open Access"                   "Source"                       
[45] "EID"                           "abstract_lower"               
[47] "keywords_lower"                "usa_IA"                       
[49] "IA_keywords"                   "IA_abstract"                  
[51] "doi_clean"                     "title_clean"

In [ ]:
df_cdPares <- df_ia_sem_duplicatas |>
  mutate(CD_Revisado_por_Pares = case_when(
    str_to_lower(`Document Type`) %in% str_to_lower(c("Article", "Review", "Conference Paper")) ~ "Sim",
    TRUE ~ "Talvez"  # outros tipos
  )) |>
  select(Title, Year, Abstract, `Document Type`,CD_Revisado_por_Pares)

df_cdPares |>
  count(`Document Type`,CD_Revisado_por_Pares)

Document Type,CD_Revisado_por_Pares,n
<chr>,<chr>,<int>
Article,Sim,137
Book chapter,Talvez,1
Conference paper,Sim,151
Conference review,Talvez,12
Data paper,Talvez,1
Editorial,Talvez,1
Review,Sim,7
Short survey,Talvez,1


### Criterio de Decisao: CD_AbstractOK

In [ ]:
df_cd2 |>
  mutate(row_id = row_number()) |>
  relocate(row_id, .before = 1) |>
  filter(
    is.na(Abstract) |
    str_squish(Abstract) == "" |
    str_detect(
      str_to_lower(Abstract),
      'no\\s+abstract' #ou, se não quise usar o padrão regex, simplesmente escrever fixed("no abstract") após a vírgula
    )
  )


row_id,Title,Year,Abstract,CD_Year_2000_2025,CD_English
<int>,<chr>,<dbl>,<chr>,<chr>,<chr>
295,Real-time in-network distribution system monitoring to improve operational efficiency,2011,[No abstract available],sim,sim


### Exportar uma matriz para abrir no google sheets e preencher os demais critérios de exclusão/inclusão de acordo com os resumos

In [ ]:
df_cd2_a <- df_cd2 |>
  mutate(CD_AbstractOK = NA_character_)

In [ ]:
write_csv(df_cd2_a,'artigos_abstractOK_scopus2.csv')